# Objective

# Init notebook

In [32]:
import requests
from bs4 import BeautifulSoup
import re
import pygsheets
import pandas as pd

keypath = 'C:\\Users\Kaspe\Documents\AuthKeys\superfoods-383120-9880791a7067.json'
gc = pygsheets.authorize(service_file=keypath)
sh = gc.open("Super Foods")

# Defining the scraping method

In [ ]:
def CleanPrice(numElement):
    numElement = numElement.get_text().strip().replace(u'\xa0', u' ')
    numElement = numElement.replace(u',', u'.')
    x = re.findall("\d+\.\d+", numElement)
    if(x):
        numElement = re.findall("\d+\.\d+", numElement)[0]
        return numElement
    else:
        return 0

In [38]:
def InsertIcaProduct(df, url):
    response = requests.get(url)
    soup_data = BeautifulSoup(response.text, 'html.parser')
   
    title  = soup_data.findAll('h1', {"class": "heading__Base-sc-1vuwqc7-0-h1 sc-ehCIER ZBWKl rZSBo"})
    title = title[0].text.strip()

    kg_price = soup_data.findAll('span', {"class": "text__Text-sc-6l1yjp-0 bhymDA"})
    if(kg_price):
        kg_price = CleanPrice(kg_price[1])
    
    price = soup_data.findAll('div', {"class" : "spacing__Spacing-sc-ngu0v9-0 lbarHM"})
    if(price):
        price = CleanPrice(price[0])
    
    contents = soup_data.findAll('div', {"class": "static-content-wrapper__StaticContentWrapper-sc-3z5iao-0 fdAbME"})
    contents.reverse()

    content_ls = contents[0].text.strip().split(",")
    key_words = ['kcal', 'kJ','Fett','mättat','Kolhydrat','socker','Fiber','Protein','Salt']
    
    for idx in range(len(key_words)):
        if(key_words[idx] not in contents[0].text):
            content_ls.insert(idx, '0')
        if(content_ls[idx]):
            x = re.findall(r'\d+(?:\.\d+)?', content_ls[idx])
            if(x):
                content_ls[idx] = x[0]

    df.loc[len(df)] = [title, content_ls[0],content_ls[1],content_ls[2],content_ls[3],content_ls[4],content_ls[5],content_ls[6],content_ls[7],content_ls[8], kg_price, price, url]
    return [df]

# Scraping

In [39]:
df = pd.DataFrame(columns=['product','kcal', 'kj', 'fat_g','sat_fat_g', 'carb_g', 'sugar_g', 'fiber_g','protein_g','salt_g', 'kg_price_kr', 'price_kr','link'])

for value in sh[1].get_all_values():
    if(value[0]!=''):
        InsertIcaProduct(df, value[0])

df.head()

,product,kcal,kj,fat_g,sat_fat_g,carb_g,sugar_g,fiber_g,protein_g,salt_g,kg_price_kr,price_kr,link
0,Tortiglioni 400g Zeta,367,1555,1.5,0.5,73,3,2.5,14,0,57.38,22.95,https://handlaprivatkund.ica.se/stores/1003417...
1,Kycklingfilé Fryst 1kg Kronfågel,90,400,1.5,0.5,0.5,0.5,0,19,0.6,124.95,124.95,https://handlaprivatkund.ica.se/stores/1003417...
2,Tofu Naturell Ekologisk 400g YiPin,118,494,5.9,1.1,2.1,0.2,0,14.2,0.01,84.88,33.95,https://handlaprivatkund.ica.se/stores/1003417...
3,Tofu naturell 400g ICA Basic,116,483,5.8,1,0.5,0.5,0,15,0.03,59.88,23.95,https://handlaprivatkund.ica.se/stores/1003417...
4,Tofu Naturell Ekologisk 270g KRAV Kung Markatta,148,615,8.6,1.4,1.6,0.1,1.7,15,0.1,110.93,29.95,https://handlaprivatkund.ica.se/stores/1003417...


# Saving results

In [40]:
# Export to google sheet and local csv 
df.to_csv('scraped_foods.csv')

wks = sh[0]
wks.set_dataframe(df,(1,1))